In [1]:
from pyalgotrade import strategy
from pyalgotrade.technical import ma
from pyalgotrade import strategy
from pyalgotrade.barfeed import csvfeed
from pyalgotrade.bar import Frequency
from pyalgotrade import plotter
import numpy as np
from pyalgotrade import broker
from pyalgotrade.barfeed import quandlfeed
import datetime
from pyalgotrade.technical import ma
filename = "nifty5min.csv"
import pytz
firstDate = datetime.datetime(2012,5,30,9,15,0,0,pytz.utc)
endDate = datetime.datetime(2016,12,9,15,25,0,0,pytz.utc)

In [2]:
def safe_round(value, digits):
    if value is not None:
        value = round(value, digits)
    return value

In [3]:
from pyalgotrade import strategy
from pyalgotrade.technical import ma
from pyalgotrade.technical import cross

In [4]:
class SMACrossOver(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument):
        strategy.BacktestingStrategy.__init__(self, feed, 100000)
        self.__instrument = instrument
        self.__position = None
        # We'll use adjusted close values instead of regular close values.
        #self.setUseAdjustedValues(False)
        #self.__prices = feed[instrument].getPriceDataSeries()
        #self.__sma = ma.SMA(self.__prices, smaPeriod)
        self.__sma10 = ma.SMA(feed[instrument].getCloseDataSeries(), 10)
        self.__sma20 = ma.SMA(feed[instrument].getCloseDataSeries(), 20)

    def getSMA(self):
        return self.__sma10
    
    def getSMA(self):
        return self.__sma20
    
    def onEnterOk(self, position):
        execInfo = position.getEntryOrder().getExecutionInfo()
        self.info("BUY at %.2f" % (execInfo.getPrice()))
        self.__position.exitStop(execInfo.getPrice() * 0.95, True)
    

    def onEnterCanceled(self, position):
        self.__position = None

    def onExitOk(self, position):
        execInfo = position.getExitOrder().getExecutionInfo()
        #self.info("SELL at %.2f" % (execInfo.getPrice()))
        #self.__position = None
        if position.getExitOrder().getType() == broker.Order.Type.STOP:
            order_type = "stop"
        else:
            order_type = "market"
        self.info("SELL at $%.2f with a %s order" % (execInfo.getPrice(), order_type))
        self.__position = None
        

    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        self.__position.exitMarket()

    def onBars(self, bars):
        # If a position was not opened, check if we should enter a long position.
        bar = bars[self.__instrument]
        if self.__position is None :
            if cross.cross_above(self.__sma10,self.__sma20) > 0 and (bar.getDateTime().hour>10 and bar.getDateTime().hour<14):
                shares = int(self.getBroker().getCash() * 0.9 / bars[self.__instrument].getPrice())
                #self.info(bar.getDateTime()) 
                # Enter a buy market order. The order is good till canceled.
                self.__position = self.enterLong(self.__instrument, shares, True)
        # Check if we have to exit the position.
        elif not self.__position.exitActive() and cross.cross_below(self.__sma10,self.__sma20) > 0:
            self.__position.exitMarket()
        elif self.__position.getAge().days > 10:
            self.__position.cancelExit()
            

In [5]:
def run_strategy():
    # Load the bar feed from the CSV file
    #feed = quandlfeed.Feed()
    #feed.addBarsFromCSV("yes_bank", "yes_bankk.csv")

    # Evaluate the strategy with the feed.
  
    feed = csvfeed.GenericBarFeed(Frequency.MINUTE,pytz.utc)
#feed = quandlfeed.Feed()
    feed.setBarFilter(csvfeed.DateRangeFilter(firstDate,endDate))
    feed.addBarsFromCSV('bank_nifty_15min',filename)

    #myStrategy = MyStrategy(feed, "bank_nifty")
    #myStrategy.run()
    crossover = SMACrossOver(feed,"bank_nifty_15min")
    crossover.run()
    print("Final portfolio value: %.2f" % crossover.getBroker().getEquity())

In [6]:
run_strategy()

2012-05-30 11:45:00+00:00 strategy [INFO] BUY at 9540.00
2012-06-04 09:20:00+00:00 strategy [INFO] SELL at $9063.00 with a stop order
2012-06-04 13:15:00+00:00 strategy [INFO] BUY at 9194.45
2012-06-15 13:20:00+00:00 strategy [INFO] SELL at $10011.50 with a market order
2012-06-18 12:55:00+00:00 strategy [INFO] BUY at 9771.55
2012-06-29 13:00:00+00:00 strategy [INFO] SELL at $10294.90 with a market order
2012-06-29 13:35:00+00:00 strategy [INFO] BUY at 10293.00
2012-07-10 13:40:00+00:00 strategy [INFO] SELL at $10659.80 with a market order
2012-07-10 13:55:00+00:00 strategy [INFO] BUY at 10696.00
2012-07-23 09:20:00+00:00 strategy [INFO] SELL at $10409.85 with a market order
2012-07-25 12:15:00+00:00 strategy [INFO] BUY at 10268.45
2012-08-06 09:20:00+00:00 strategy [INFO] SELL at $10504.00 with a market order
2012-08-06 12:25:00+00:00 strategy [INFO] BUY at 10538.85
2012-08-17 12:30:00+00:00 strategy [INFO] SELL at $10537.00 with a market order
2012-08-21 11:25:00+00:00 strategy [INFO

2014-03-18 11:45:00+00:00 strategy [INFO] BUY at 12265.35
2014-03-31 09:20:00+00:00 strategy [INFO] SELL at $12930.00 with a market order
2014-03-31 11:50:00+00:00 strategy [INFO] BUY at 12872.00
2014-04-11 11:55:00+00:00 strategy [INFO] SELL at $12905.00 with a market order
2014-04-11 12:50:00+00:00 strategy [INFO] BUY at 12954.10
2014-04-22 12:55:00+00:00 strategy [INFO] SELL at $12990.00 with a market order
2014-04-28 11:20:00+00:00 strategy [INFO] BUY at 13077.95
2014-05-09 11:25:00+00:00 strategy [INFO] SELL at $13360.00 with a market order
2014-05-12 11:05:00+00:00 strategy [INFO] BUY at 14158.95
2014-05-23 11:10:00+00:00 strategy [INFO] SELL at $15088.05 with a market order
2014-05-23 11:55:00+00:00 strategy [INFO] BUY at 15151.00
2014-06-03 12:00:00+00:00 strategy [INFO] SELL at $15247.05 with a market order
2014-06-03 12:05:00+00:00 strategy [INFO] BUY at 15324.35
2014-06-16 09:20:00+00:00 strategy [INFO] SELL at $15070.00 with a market order
2014-06-16 13:35:00+00:00 strategy

2016-02-10 12:15:00+00:00 strategy [INFO] BUY at 14643.70
2016-02-12 09:20:00+00:00 strategy [INFO] SELL at $13911.52 with a stop order
2016-02-12 13:10:00+00:00 strategy [INFO] BUY at 14013.95
2016-02-23 13:15:00+00:00 strategy [INFO] SELL at $14017.95 with a market order
2016-02-24 12:10:00+00:00 strategy [INFO] BUY at 13858.00
2016-03-08 09:20:00+00:00 strategy [INFO] SELL at $15153.95 with a market order
2016-03-08 13:10:00+00:00 strategy [INFO] BUY at 15188.80
2016-03-21 09:20:00+00:00 strategy [INFO] SELL at $15758.30 with a market order
2016-03-21 12:35:00+00:00 strategy [INFO] BUY at 15821.00
2016-04-01 12:40:00+00:00 strategy [INFO] SELL at $16123.95 with a market order
2016-04-01 13:45:00+00:00 strategy [INFO] BUY at 16162.95
2016-04-12 13:50:00+00:00 strategy [INFO] SELL at $15857.25 with a market order
2016-04-13 12:50:00+00:00 strategy [INFO] BUY at 16338.20
2016-04-25 09:20:00+00:00 strategy [INFO] SELL at $16721.75 with a market order
2016-04-27 12:05:00+00:00 strategy [

Final portfolio value: 144241.06
